In [54]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver #(conda install -c conda-forge selenium)

In [55]:
r = requests.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats')
r.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr"\n  xmlns:og="http://ogp.me/ns#"\n  xmlns:article="http://ogp.me/ns/article#"\n  xmlns:book="http://ogp.me/ns/book#"\n  xmlns:profile="http://ogp.me/ns/profile#"\n  xmlns:video="http://ogp.me/ns/video#"\n  xmlns:product="http://ogp.me/ns/product#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta name="apple-itunes-app" content="app-id=578411515" />\n<link rel="shortcut icon" href="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/favicon.ico" type="image/vnd.microsoft.icon" />\n<meta name="twitter:card" content="summary_large_image" />\n<meta name="twitter:image" content="https://www.timeshighereducation.com/sites/default/themes/custom/the_res

In [57]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.h1

<h1 class="pane-title">World University Rankings 2018</h1>

In [ ]:
table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="datatable-1") 
rows = table.tbody.findAll(lambda tag: tag.name=='tr')

In [ ]:
body = table.find('thead')
body

In [59]:
from selenium import webdriver

browser = webdriver.Chrome()  
browser.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats')  
html_source = browser.page_source  
browser.quit()

soup = BeautifulSoup(html_source, 'html.parser')  
table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="datatable-1") 
rows = table.tbody.findAll(lambda tag: tag.name=='tr')
print(rows)

[<tr class="odd" data-nid="468" role="row"><td class="rank sorting_1 sorting_2">1</td><td class=" name namesearch"><a class="ranking-institution-title" data-mz="" data-position="title" href="/world-university-rankings/university-oxford#ranking-dataset/629337">University of Oxford</a><div class="location"><span><i class="fa fa-map-marker"></i>United Kingdom</span></div><a class="btn btn-gray btn-margin" data-mz="" data-position="explore" href="/world-university-rankings/university-oxford#ranking-dataset/629337">Explore</a></td><td class=" stats stats_number_students">20,409</td><td class=" stats stats_student_staff_ratio">11.2</td><td class=" stats stats_pc_intl_students">38%</td><td class=" stats stats_female_male_ratio">46 : 54</td></tr>, <tr class="even" data-nid="470" role="row"><td class="rank sorting_1 sorting_2">2</td><td class=" name namesearch"><a class="ranking-institution-title" data-mz="" data-position="title" href="/world-university-rankings/university-cambridge#ranking-dat

In [90]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [91]:
SITE = "https://www.timeshighereducation.com"
URL = SITE+"/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"

In [92]:
N = 200
r = requests.get(URL)
all_data = r.json()
print(list(all_data.keys()))
data = all_data.get("data")[:N]
print(list(data[0].keys()))

['pillars', 'data', 'subjects', 'locations']
['member_level', 'location', 'subjects_offered', 'scores_research', 'stats_student_staff_ratio', 'scores_citations_rank', 'scores_international_outlook', 'rank', 'scores_international_outlook_rank', 'scores_citations', 'scores_industry_income', 'url', 'record_type', 'stats_pc_intl_students', 'scores_teaching', 'aliases', 'scores_teaching_rank', 'scores_research_rank', 'scores_overall_rank', 'stats_female_male_ratio', 'scores_industry_income_rank', 'nid', 'rank_order', 'stats_number_students', 'scores_overall', 'name']


name, rank, country and region, number of faculty members (international and total) and number of students (international and total)

In [93]:
data[0]

{'aliases': 'University of Oxford',
 'location': 'United Kingdom',
 'member_level': '0',
 'name': 'University of Oxford',
 'nid': 468,
 'rank': '1',
 'rank_order': '10',
 'record_type': 'master_account',
 'scores_citations': '99.1',
 'scores_citations_rank': '15',
 'scores_industry_income': '63.7',
 'scores_industry_income_rank': '169',
 'scores_international_outlook': '95.0',
 'scores_international_outlook_rank': '24',
 'scores_overall': '94.3',
 'scores_overall_rank': '10',
 'scores_research': '99.5',
 'scores_research_rank': '1',
 'scores_teaching': '86.7',
 'scores_teaching_rank': '5',
 'stats_female_male_ratio': '46 : 54',
 'stats_number_students': '20,409',
 'stats_pc_intl_students': '38%',
 'stats_student_staff_ratio': '11.2',
 'subjects_offered': 'Archaeology,Art, Performing Arts & Design,Biological Sciences,Business & Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics & Econometrics,Electrical & Electronic Engineering,General Engineering,Geo

In [94]:
s = requests.Session()
reqs = [(req_id, s.prepare_request(requests.Request('GET', SITE+entry["url"]))) for req_id, entry in enumerate(data)]
print(len(reqs), "requests to be sent.")

resps = [(req_id, s.send(req)) for req_id, req in reqs]

done = [(req_id, resp.text) for req_id, resp in resps if resp.status_code == 200]
failed = [(req_id, resp) for req_id, resp in resps if resp.status_code != 200]

print("%d done, %d failed." % (len(done), len(failed)))

200 requests to be sent.
200 done, 0 failed.


In [95]:
uni = pd.DataFrame(data, columns=["name", "rank", "location", "stats_pc_intl_students", "stats_student_staff_ratio", "url"])
uni.rename(columns={"location":"country","stats_pc_intl_students":"pc_intl_students(%)", "stats_student_staff_ratio":"student_staff_ratio"},inplace=True)

In [96]:
uni["rank"] = uni["rank"].str.extract('(\d+)', expand=False).astype(int)
uni["pc_intl_students(%)"]= uni["pc_intl_students(%)"].str.extract('(\d+)', expand=False).astype(int)

In [97]:
uni

,name,rank,country,pc_intl_students(%),student_staff_ratio,url
0,University of Oxford,1,United Kingdom,38,11.2,/world-university-rankings/university-oxford
1,University of Cambridge,2,United Kingdom,35,10.9,/world-university-rankings/university-cambridge
2,California Institute of Technology,3,United States,27,6.5,/world-university-rankings/california-institut...
3,Stanford University,3,United States,22,7.5,/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,5,United States,34,8.7,/world-university-rankings/massachusetts-insti...
5,Harvard University,6,United States,26,8.9,/world-university-rankings/harvard-university
6,Princeton University,7,United States,24,8.3,/world-university-rankings/princeton-university
7,Imperial College London,8,United Kingdom,55,11.4,/world-university-rankings/imperial-college-lo...
8,University of Chicago,9,United States,25,6.2,/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,10,Switzerland,38,14.6,/world-university-rankings/eth-zurich-swiss-fe...
